In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import time

In [6]:
cmc = requests.get('https://coinmarketcap.com/')
soup = BeautifulSoup(cmc.content, 'html.parser')

In [7]:
soup.title

<title>Cryptocurrency Prices, Charts And Market Capitalizations | CoinMarketCap</title>

In [8]:
# This returns an organized version of CoinMarketCap and I’m able to eyeball and locate the data that I want
a = soup.prettify()

In [12]:
# I know that bitcoin’s page is formatted as https://coinmarketcap.com/currencies/bitcoin/
# This matches the key ‘slug’ so at a minimum, I know that I’ll need to save slug. 
# I’ll also save id as well just in case. 
# I also see that this is JSON data within a script tag so we’ll need to isolate it.

data = soup.find('script', id="__NEXT_DATA__", type="application/json")

coins = {}

#using data.contents[0] to remove script tags
coin_data = json.loads(data.contents[0])
listings = coin_data['props']['initialState']['cryptocurrency']['listingLatest']['data']

attributes = listings[0]["keysArr"]
index_of_id = attributes.index("id")
index_of_slug = attributes.index("slug")

for i in listings[1:]:
    coins[str(i[index_of_id])] = i[index_of_slug]



In [13]:
# the template for viewing these tables for any coin by date goes: 
# https://coinmarketcap.com/currencies/[slug]/historical-data/?start=[YYYYMMDD]&end=[YYYYMMDD]

for i in coins:
    page = requests.get(f'https://coinmarketcap.com/currencies/{coins[i]}/historical-data/?start=20200101&end=20200630')
    soup = BeautifulSoup(page.content, 'html.parser')
    data = soup.find('script', id="__NEXT_DATA__", type='application/json')
    historical_data = json.loads(data.contents[0])

In [16]:
# quotes = historical_data['props']['initialState']['cryptocurrency']['ohlcvHistorical'][i]['quotes']
# info = historical_data['props']['initialState']['cryptocurrency']['ohlcvHistorical'][i]

In [17]:
coin_name = []
coin_symbol = []
market_cap = []
percent_change_1h = []
percent_change_24h = []
percent_change_7d = []
price = []
volume_24h = []

index_of_slug = attributes.index("slug")
index_of_symbol = attributes.index("symbol")

In [24]:
sel_currency = 'USD'
index_of_quote_currency_price = attributes.index(
    f"quote.{sel_currency}.price"
)
index_of_quote_currency_percent_change_1h = attributes.index(
    f"quote.{sel_currency}.percentChange1h"
)
index_of_quote_currency_percent_change_24h = attributes.index(
    f"quote.{sel_currency}.percentChange24h"
)
index_of_quote_currency_percent_change_7d = attributes.index(
    f"quote.{sel_currency}.percentChange7d"
)
index_of_quote_currency_market_cap = attributes.index(
    f"quote.{sel_currency}.marketCap"
)
index_of_quote_currency_volume_24h = attributes.index(
    f"quote.{sel_currency}.volume24h"
)
for i in listings[1:]:
    coin_name.append(i[index_of_slug])
    coin_symbol.append(i[index_of_symbol])

    price.append(i[index_of_quote_currency_price])
    percent_change_1h.append(i[index_of_quote_currency_percent_change_1h])
    percent_change_24h.append(i[index_of_quote_currency_percent_change_24h])
    percent_change_7d.append(i[index_of_quote_currency_percent_change_7d])
    market_cap.append(i[index_of_quote_currency_market_cap])
    volume_24h.append(i[index_of_quote_currency_volume_24h])

In [25]:
df = pd.DataFrame(
    columns=[
        "coin_name",
        "coin_symbol",
        "market_cap",
        "percent_change_1h",
        "percent_change_24h",
        "percent_change_7d",
        "price",
        "volume_24h",
    ]
)
df["coin_name"] = coin_name
df["coin_symbol"] = coin_symbol
df["price"] = price
df["percent_change_1h"] = percent_change_1h
df["percent_change_24h"] = percent_change_24h
df["percent_change_7d"] = percent_change_7d
df["market_cap"] = market_cap
df["volume_24h"] = volume_24h


In [26]:
df

,coin_name,coin_symbol,market_cap,percent_change_1h,percent_change_24h,percent_change_7d,price,volume_24h
0,bitcoin,BTC,7.517312e+11,0.478514,1.835330,2.581629,39616.094319,1.642559e+10
1,ethereum,ETH,3.201769e+11,0.175808,2.444547,-2.239498,2671.683748,7.629531e+09
2,tether,USDT,7.972845e+10,-0.007607,-0.010219,-0.029658,1.000193,3.772041e+10
3,bnb,BNB,6.384327e+10,0.368726,4.502405,5.499065,386.655306,1.249326e+09
4,usd-coin,USDC,5.290800e+10,0.040908,0.070254,0.037836,0.999871,2.569098e+09
...,...,...,...,...,...,...,...,...
195,compound,COMP,7.285720e+08,0.514364,3.314575,0.409192,110.238278,5.473768e+07
196,iotex,IOTX,7.028427e+08,1.325157,5.421888,2.639666,0.073667,1.938989e+07
197,sxp,SXP,6.840381e+08,1.079008,4.407527,13.343349,1.425265,6.920441e+07
198,xinfin,XDC,6.171798e+08,-0.461317,0.914114,-5.037492,0.050157,7.560916e+06


In [27]:
df.to_csv('criptoes.csv', index = False)